# 1.온라인 선호지수 분석용 데이터 생성
## Search1과 Custom에 공통적으로 포함된 Client Id만으로 데이터프레임 생성
### Developing Online Preference based on Client Selection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [2]:
#데이터가져오기
Product = pd.read_csv("C:/Users/lsh93/Desktop/data/Product.csv")
Search1 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search1.csv")
Search2 = pd.read_csv("C:/Users/lsh93/Desktop/data/Search2.csv")
Custom = pd.read_csv("C:/Users/lsh93/Desktop/data/Custom.csv")
Session = pd.read_csv("C:/Users/lsh93/Desktop/data/Session.csv")
Master = pd.read_csv("C:/Users/lsh93/Desktop/data/Master.csv")

C:\Users\lsh93\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#구매데이터의 client 수 != 고객데이터의 client 수 
#구매데이터의 client 수 != 검색데이터의 client 수
print('Product shape: ',Product.shape)
print('Custom shape: ',Custom.shape)
print('Search1 shape: ',Search1.shape)
print('Session shape: ',Session.shape)
print()
print('구매에 참여한 client 수: ',len(Product['CLNT_ID'].value_counts()))
print('고객데이터에 있는 unique한 client 수: ',len(Custom['CLNT_ID'].value_counts()))
print('세션에 참여한 unique한 client 수: ',len(Session['CLNT_ID'].value_counts()))
print('검색에 참여한 client 수: ', len(Search1['CLNT_ID'].value_counts()))

Product shape:  (5024906, 8)
Custom shape:  (671679, 3)
Search1 shape:  (2884943, 4)
Session shape:  (2712907, 9)

구매에 참여한 client 수:  922737
고객데이터에 있는 unique한 client 수:  671679
세션에 참여한 unique한 client 수:  922737
검색에 참여한 client 수:  511477


- 세션에 참여한 client 수와 구매에 참여한 client 수는 동일하다 
- 상품을 구매했을지라도 검색을 하지 않은 client들이 있다 ) 922737 - 511477 만큼
- 고객데이터에 모든 client들의 정보가 있는 것은 아니다 

In [4]:
temp1 = pd.merge(Product,Session, how = 'inner' , on = 'CLNT_ID')

In [5]:
print(len(temp1['CLNT_ID'].value_counts()))

922737


In [6]:
temp2 = pd.merge(Custom, Product, how = 'right' , on = 'CLNT_ID')

In [7]:
print(len(temp1['CLNT_ID'].value_counts()))

922737


In [8]:
#null값 체크 --> 이건 unique한 건 아니지만 그래도 null값이 있다는 걸 의미함 
print(temp2['CLNT_AGE'].isnull().sum().sum())

1036319


# Product Outlier 제거
- 데이터 전처리 : outlier 제거
- HIT_SEQ가 1인 것 제거하기 

In [9]:
#HIT_SEQ가 1인 row 제거하기 (outlier로 간주하라고 하였음)
print('제거 전: ',len(Product['CLNT_ID'].value_counts()))  
idx = (Product['HITS_SEQ'] != 1) 
Product = Product[idx] 
print('제거 후: ',len(Product['CLNT_ID'].value_counts()))  

제거 전:  922737
제거 후:  922618


In [10]:
Product[Product['CLNT_ID'] == 4758889]  

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
533815,4758889,10607258,24,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1
597057,4758889,10607258,195,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1
655687,4758889,10607258,298,587541,색상:블랙|사이즈:F / 1개,비키,"45,000",1
661037,4758889,10730334,76,692624,색상:블랙 / 1개,LG전자,"45,340",1
4954520,4758889,10607258,298,776358,사이즈:28 / 1개,게스(진),"39,000",1
5022807,4758889,10607258,195,776358,사이즈:28 / 1개,게스(진),"39,000",1


## 1.1 상품 구매량 count를 위한 테이블 만들기

- 상품 구매 count를 하기 위해서, Product에서 HIT_SEQ = 1을 제거하고(앞에서 이미 제거됨), Search1과 custom에 둘 다 존재하는 Client 기준으로 정렬
- 한 client의 hit번호가 가장 큰 것에 대해서만 따로 빼서 매트릭스를 만들고, 그 후 상품분류에 대해 value_counts()해 주면 될듯

In [11]:
#Search1과 custom의 공통된 client id 뽑고, unique하게 만들어주기
Client = pd.merge(Search1[['CLNT_ID']], Custom[['CLNT_ID']])
Unique_Client = Client.drop_duplicates(subset=None, keep='last', inplace=False)

In [12]:
Unique_Client['CLNT_ID'].value_counts()

5461714    1
557637     1
5280327    1
5328413    1
2685514    1
6353483    1
4246092    1
4768333    1
55886      1
2150991    1
1583696    1
1581649    1
5767765    1
5773910    1
2365283    1
1079898    1
553563     1
5784157    1
6314590    1
6400608    1
2380969    1
2136646    1
3191363    1
4842089    1
2772219    1
4954662    1
2855463    1
2353704    1
1303081    1
5857480    1
          ..
4580630    1
2987288    1
2465049    1
4762753    1
6130971    1
1081954    1
4589347    1
5615932    1
4015362    1
4740254    1
1896362    1
1211621    1
6448358    1
1532795    1
5179014    1
3592504    1
4859118    1
3331312    1
5954801    1
6487285    1
5141990    1
4908278    1
6505990    1
2266360    1
5938425    1
6448371    1
4371708    1
3319038    1
6466815    1
6291456    1
Name: CLNT_ID, Length: 375864, dtype: int64

In [13]:
#Search1과 custom에 공통으로 있는 client id에 해당하는 구매내역만 dataframe으로 만들기 
Product_Client = pd.merge(Product, Unique_Client, how = 'inner', on = 'CLNT_ID')
print(len(Product_Client['CLNT_ID'].value_counts()))   

375864


In [14]:
#Product_Client = Product_Client.drop_duplicates(subset=None, keep='last', inplace=False)
#print(len(Product_Client['CLNT_ID'].value_counts()))   

In [15]:
Product_Client[Product_Client['CLNT_ID']==4140076] 

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2
1,4140076,10342512,23,422358,선택:02.204143-02S|사이즈:M5W7(240) / 1개,크록스,"39,900",1
2,4140076,6526651,22,196272,색상:선택3)248(네이비)|사이즈:L(105) / 1개,프레드페리,"47,600",1
3,4140076,8326529,199,305965,사이즈:115_3XL / 1개,아디다스(의류),"46,700",1
4,4140076,8891563,19,305965,사이즈:115_3XL / 1개,아디다스(의류),"46,700",1
5,4140076,6526651,193,196272,색상:선택2)102(블랙)|사이즈:XL(110) / 1개,프레드페리,"47,600",1
6,4140076,6591360,149,196272,색상:선택2)102(블랙)|사이즈:XL(110) / 2개,프레드페리,"47,600",2
7,4140076,6526651,22,365025,색상:WH|사이즈:165 / 1개,닥스 키즈,"48,300",1
8,4140076,4756252,147,794908,1개,네이쳐리빙,"7,200",1
9,4140076,10342512,23,403243,사이즈:M9 / 1개,크록스,"57,000",1


## 1.2 Client Behavior 데이터 만들기

#### Client의 행동을 따라가보자
- Client가 어떻게 행동하는지 데이터 살펴보기
- Client Behavior 데이터 만들기 

- Client ID, 날짜(세션일자), 세션 ID, HIT번호, 검색키워드, 검색건수, 상품명, 구매건수 

- CLNT_ID(Product), SESS_DT(Session), SESS_ID(세션ID), HITS_SEQ(Product)
- KWD_NM(Search1), SEARCH_CNT(Search1), PD_NM(Master), PD_BUY_CT(Product)

#### Search1의 client < custom의 client < Product의 client = Session의 Client
#### Search1의 client 중에 custom 정보 없는 것 있다. 
#### 검색을 하고, custom 정보가 있는 client만을 기준으로 table 작성 (375864개의 client id)

In [16]:
Product_Client.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2
1,4140076,10342512,23,422358,선택:02.204143-02S|사이즈:M5W7(240) / 1개,크록스,"39,900",1
2,4140076,6526651,22,196272,색상:선택3)248(네이비)|사이즈:L(105) / 1개,프레드페리,"47,600",1
3,4140076,8326529,199,305965,사이즈:115_3XL / 1개,아디다스(의류),"46,700",1
4,4140076,8891563,19,305965,사이즈:115_3XL / 1개,아디다스(의류),"46,700",1


In [17]:
data1 = pd.merge(Product_Client, Session)
print(len(data1['CLNT_ID'].value_counts()))

375864


In [18]:
#상품정보를 추가해준다
data2 = pd.merge(data1, Master, how = 'inner', on = 'PD_C')
print(len(data2['CLNT_ID'].value_counts()))

375864


In [19]:
#세션아이디도 동일한 것으로
#client -> 검색 -> 구매
Client_Behavior = pd.merge(data2, Search1, on=['CLNT_ID', 'SESS_ID'])
print(len(Client_Behavior['CLNT_ID'].value_counts()))
print('CLNT_ID랑 SESS_ID가 동일해야 하는 조건 때문에 client 수 약간 줄어들었음.')

375839
CLNT_ID랑 SESS_ID가 동일해야 하는 조건 때문에 client 수 약간 줄어들었음.


In [20]:
Client_Behavior

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,...,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,KWD_NM,SEARCH_CNT
0,4148106,6662803,94,570603,색상:WHT0_WHT0-(WHITE)|사이즈:115 / 1개,데상트,"39,000",1,55,20180611,...,"1,371",mobile,Seoul,Seoul,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,라코스떼 남성티셔츠,2
1,4148106,6662803,94,529438,색상:선1)001(화이트)|사이즈:006(110) / 1개,라코스테(의류),"88,000",1,55,20180611,...,"1,371",mobile,Seoul,Seoul,남성 빅크록 라운드 반팔 티셔츠TH670E-18B_6컬러 - 선1)001(화이트) ...,남성의류,남성의류상의,남성티셔츠,라코스떼 남성티셔츠,2
2,5298612,4443505,129,570603,색상:BLK0_(BLK0)BLACK|사이즈:105 / 1개,데상트,"39,000",1,710,20180717,...,"1,199",mobile,Busan,Busan,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,남성반바지,1
3,5298612,4443505,129,570603,색상:BLK0_(BLK0)BLACK|사이즈:105 / 1개,데상트,"39,000",1,710,20180717,...,"1,199",mobile,Busan,Busan,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,데상트 반팔,1
4,5639922,5456967,91,570603,색상:BLK0_(BLK0)BLACK|사이즈:95 / 1개,데상트,"39,000",1,1409,20180701,...,"1,598",mobile,Gyeongsangbuk-do,Chilgok-gun,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,르꼬끄반팔,1
5,5639922,5456967,91,570603,색상:BLK0_(BLK0)BLACK|사이즈:95 / 1개,데상트,"39,000",1,1409,20180701,...,"1,598",mobile,Gyeongsangbuk-do,Chilgok-gun,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,르꼬끄여성바지,1
6,5639922,5456967,91,262991,색상:BLK0_(BLK0)BLACK|사이즈:090 / 1개,르꼬끄,"39,000",1,1409,20180701,...,"1,598",mobile,Gyeongsangbuk-do,Chilgok-gun,[본사직영] 남녀공용 선데이 변형 반팔 티셔츠 (Q8223ARS42) - BLK0_...,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,르꼬끄반팔,1
7,5639922,5456967,91,262991,색상:BLK0_(BLK0)BLACK|사이즈:090 / 1개,르꼬끄,"39,000",1,1409,20180701,...,"1,598",mobile,Gyeongsangbuk-do,Chilgok-gun,[본사직영] 남녀공용 선데이 변형 반팔 티셔츠 (Q8223ARS42) - BLK0_...,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,르꼬끄여성바지,1
8,5639922,5456967,72,262991,색상:BLK0_(BLK0)BLACK|사이즈:090 / 1개,르꼬끄,"39,000",1,1409,20180701,...,"1,598",mobile,Gyeongsangbuk-do,Chilgok-gun,[본사직영] 남녀공용 선데이 변형 반팔 티셔츠 (Q8223ARS42) - BLK0_...,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,르꼬끄반팔,1
9,5639922,5456967,72,262991,색상:BLK0_(BLK0)BLACK|사이즈:090 / 1개,르꼬끄,"39,000",1,1409,20180701,...,"1,598",mobile,Gyeongsangbuk-do,Chilgok-gun,[본사직영] 남녀공용 선데이 변형 반팔 티셔츠 (Q8223ARS42) - BLK0_...,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠,르꼬끄여성바지,1


- Hit번호만 다르고 나머지 element의 내용이 같은 row가 여러 개가 있는 경우(중복 행) -> Hit번호가 가장 큰 row만 남긴다 

# 2. 온라인 선호지수 개발

1) 구매선호지수(client i, 상품군 j)
- Product 데이터를 이용(실제 구매 데이터), 다른 item 대비 특정 상품군을 얼마나 구매하였는가  = 특정 상품군 구매 개수 / 각 client의 총 구매 개수
- client i = 1~375839 (Product, Search1, Custom모두에 정보가 있는 Client)

In [21]:
Client_Used = Client_Behavior[['CLNT_ID']]
Client_Used = Client_Used.drop_duplicates(subset=None, keep='last', inplace=False) 

In [22]:
print(len(Client_Used['CLNT_ID'].value_counts()))

375839


In [23]:
Product_Used = pd.merge(Product, Client_Used, how = 'inner', on = 'CLNT_ID')

In [24]:
print(len(Product_Used['CLNT_ID'].value_counts()))

375839


In [25]:
#상품정보추가
Product_info = pd.merge(Product_Used, Master, how = 'inner', on = 'PD_C')

In [26]:
print(len(Product_info['CLNT_ID'].value_counts()))

375839


In [27]:
Product_info.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,"39,000",2,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠
1,4148106,6537318,21,570603,색상:BLK0_(BLK0)BLACK|사이즈:115 / 1개,데상트,"39,000",1,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠
2,4148106,6662803,94,570603,색상:WHT0_WHT0-(WHITE)|사이즈:115 / 1개,데상트,"39,000",1,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠
3,5298612,4443505,129,570603,색상:BLK0_(BLK0)BLACK|사이즈:105 / 1개,데상트,"39,000",1,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠
4,5553084,6453463,55,570603,색상:WHT0_WHT0-(WHITE)|사이즈:100 / 1개,데상트,"39,000",1,[데상트] 스포츠베이직 빅로고 티셔츠 - DMGR_DMELANGE / 115,스포츠패션,남성일반스포츠의류,남성스포츠티셔츠


- 우선은 상품군을 대분류 기준으로 지정

In [67]:
#구매선호지수 만드는 데 이 데이터 이용할 것 
Product_num1= Product_info[['CLNT_ID','PD_BUY_CT','CLAC1_NM']]

In [ ]:
#Client의 수
print('client의 수(row): ',len(Product_num1['CLNT_ID'].value_counts()))
#대분류의 수
print('대분류의 수(col): ',len(Product_num1['CLAC1_NM'].value_counts()))

- 구매선호지수값을 할당할 Purchase_Pref 데이터프레임 초기화 과정 (값은 없음)

In [279]:
#matrix를 생성해 주기 위한 작업
#Client dataframe 생성

C = Product_num1[['CLNT_ID']]
C = C.drop_duplicates(subset=None, keep='first', inplace=False)
C = C['CLNT_ID']
print(len(C))

375839


In [280]:
#대분류 dataframe생성
CL1 = Product_num1[['CLAC1_NM']]
CL1 = CL1.drop_duplicates(subset=None, keep='first', inplace=False)
CL1 = CL1['CLAC1_NM']
print(len(CL1))

37


In [281]:
#dataframe을 array로 변환
row_names = C.as_matrix()
print(row_names.shape)

(375839,)


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [282]:
col_names = CL1.as_matrix()
print(col_names.shape)

(37,)


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [283]:
matrix = np.zeros((len(C),len(CL1)))
print(matrix.shape) 

(375839, 37)


In [284]:
#구매선호지수 dataframe 기본 틀 생성 
Purchase_Pref = pd.DataFrame(matrix, columns=col_names, index=row_names)
print(Purchase_Pref.shape)

(375839, 37)


In [285]:
Purchase_Pref.columns.values

array(['스포츠패션', '남성의류', '유아동의류', '청소/세탁/욕실용품', '완구', '패션잡화', '시즌스포츠',
       '여성의류', '인테리어/조명', '속옷/양말/홈웨어', '과일', '화장품/뷰티케어', '건강식품',
       '문구/사무용품', '주방잡화', '생활/주방가전', '침구/수예', '컴퓨터', '계절가전', '가구',
       '식기/조리기구', '모바일', '헬스/피트니스', '아웃도어/레저', '퍼스널케어', '세제/위생',
       '출산/육아용품', '음료', '구기/필드스포츠', '냉장/세탁가전', '냉동식품', '냉장식품', '원예/애완',
       '상품권', '자동차용품', '축산물', '영상/음향가전'], dtype=object)

In [286]:
Purchase_Pref.index.values

array([4140076, 4148106, 5298612, ..., 4362431, 6510160, 1835621],
      dtype=int64)

#### Product_num1 -> Purchase_Pref 로 변환하는 과정 (Purchase_Pref 매트릭스 안에 값을 할당하는 과정)

In [287]:
#Product_num1의 data type확인
Product_num1.dtypes

CLNT_ID       int64
PD_BUY_CT     int64
CLAC1_NM     object
dtype: object

In [183]:
#PD_BUY_AM의 data type을 바꿔줘야 함(count를 위해서는)
d = Product_num1[['PD_BUY_CT']]
d.dtypes

PD_BUY_CT    object
dtype: object

- PD_BUY_CT(구매수량)의 data type을 바꿔줘야 함
- PD_BUY_CT 내 count에서 '1,387'과 같은 것에서 ','를 삭제해 줘야 -> for문 돌려서 일일히 삭제하고 할당해야 함

In [98]:
#에러나는 부분
#PD_BUY_CT(구매수량)의 data type을 바꿔줘야 함
a = pd.DataFrame(d, dtype='float')
a

ValueError: could not convert string to float: '1,387'

In [105]:
#','가 있는 부분에서 에러가 나서 ,를 공백으로 바꿔줘야 함 
idx = d['PD_BUY_CT'] == '1,387'
d[idx]

,PD_BUY_CT
1227284,"1,387"


In [114]:
#값 하나하나에 대해 바꿔줘야 한다는 것을 알 수 있음
c =  '1,387'.replace(",", "")
c

'1387'

In [175]:
#type가 str임 
e = d['PD_BUY_CT'][1227284] 
print(type(e))
e = e.replace(",", "")
e

<class 'str'>


'1387'

In [184]:
#int에 해당하는 아이는 replace가 안됨, str에 해당하는 것만 됨
length = len(d)
i = 1000000
e = d['PD_BUY_CT'][i].replace(",","")

AttributeError: 'int' object has no attribute 'replace'

In [190]:
type(d['PD_BUY_CT'][1000001]) == int

True

In [192]:
#PD_BUY_CT 내 count에서 '1,387'과 같은 것에서 ','를 삭제해 줘야 
for i in range(len(d)):
    if type(Product_num1['PD_BUY_CT'][i]) == str:
        Product_num1['PD_BUY_CT'][i] = Product_num1['PD_BUY_CT'][i].replace(",","")
    else:
        continue

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [199]:
#PD_BUY_CT(구매수량)의 data type을 바꿔줌 (str to int)
Product_num1['PD_BUY_CT'] = pd.DataFrame(Product_num1['PD_BUY_CT'], dtype='int64')
Product_num1[['PD_BUY_CT']]

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,PD_BUY_CT
0,2
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [200]:
#Product_num1의 data type확인(잘 바뀌었는지)
Product_num1.dtypes

CLNT_ID       int64
PD_BUY_CT     int64
CLAC1_NM     object
dtype: object

- 이제 Purchase_Pref matrix안에 값 채워넣기 

1) 각 client마다 구매한 상품 대분류(i,j)에 구매 수량(PD_BUY_CT)채워넣기

In [202]:
#데이터 형태 확인
Product_num1.head()

,CLNT_ID,PD_BUY_CT,CLAC1_NM
0,4140076,2,스포츠패션
1,4148106,1,스포츠패션
2,4148106,1,스포츠패션
3,5298612,1,스포츠패션
4,5553084,1,스포츠패션


In [214]:
#만들어놓은 기본 틀 모양 확인
print(Purchase_Pref.shape)
print('column names: \n',Purchase_Pref.columns.values)
print('Client ID: \n',Purchase_Pref.index.values)

(375839, 37)
column names: 
 [('스포츠패션',) ('남성의류',) ('유아동의류',) ('청소/세탁/욕실용품',) ('완구',) ('패션잡화',)
 ('시즌스포츠',) ('여성의류',) ('인테리어/조명',) ('속옷/양말/홈웨어',) ('과일',) ('화장품/뷰티케어',)
 ('건강식품',) ('문구/사무용품',) ('주방잡화',) ('생활/주방가전',) ('침구/수예',) ('컴퓨터',)
 ('계절가전',) ('가구',) ('식기/조리기구',) ('모바일',) ('헬스/피트니스',) ('아웃도어/레저',)
 ('퍼스널케어',) ('세제/위생',) ('출산/육아용품',) ('음료',) ('구기/필드스포츠',) ('냉장/세탁가전',)
 ('냉동식품',) ('냉장식품',) ('원예/애완',) ('상품권',) ('자동차용품',) ('축산물',) ('영상/음향가전',)]
Client ID: 
 [[4140076]
 [4148106]
 [5298612]
 ...
 [4362431]
 [6510160]
 [1835621]]


In [290]:
print(Purchase_Pref.columns[0]) 
print(Purchase_Pref.columns[2])
print(Purchase_Pref.index[0])
print(Purchase_Pref.index[2])

스포츠패션
유아동의류
4140076
5298612


In [291]:
columns = Purchase_Pref.columns
print(columns[0])
client_id = Purchase_Pref.index
print(client_id[0])

스포츠패션
4140076


In [330]:
Purchase_Pref["스포츠패션"][4140076] = 1

In [332]:
Purchase_Pref["스포츠패션"][4140076]

2.0

- Product_num1 을 Purchase_Pref에 할당함

In [346]:
print(len(Product_num1))
print(Product_num1['CLNT_ID'][1])
print(Product_num1[['CLNT_ID','PD_BUY_CT','CLAC1_NM']][0:4])

3073220
4148106
   CLNT_ID  PD_BUY_CT CLAC1_NM
0  4140076          2    스포츠패션
1  4148106          1    스포츠패션
2  4148106          1    스포츠패션
3  5298612          1    스포츠패션


In [ ]:
#값 할당
for i in range(0,1000000):
    row = Product_num1['CLNT_ID'][i] 
    col = Product_num1['CLAC1_NM'][i] 
    element = Product_num1['PD_BUY_CT'][i] 
    Purchase_Pref[col][row] = Purchase_Pref[col][row] + element 

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


range(0, 3)
